In [97]:
def divide_map_bounds(west, east, south, north, num_lon_splits, num_lat_splits):
    lon_step = (east - west) / num_lon_splits
    lat_step = (north - south) / num_lat_splits
    grids = []
    for i in range(num_lon_splits):
        for j in range(num_lat_splits):
            grid_west = west + (i * lon_step)
            grid_east = grid_west + lon_step
            grid_south = south + (j * lat_step)
            grid_north = grid_south + lat_step

            grids.append({
                'west': grid_west,
                'east': grid_east,
                'south': grid_south,
                'north': grid_north
            })

    return grids

mapBounds = {
    'west': -92.088777203125,
    'east': -67.633210796875,
    'south': 32.768894735707235,
    'north': 37.52724251059601,
}
splitted_coords = divide_map_bounds(**mapBounds, num_lon_splits=3, num_lat_splits=3)
splitted_coords

[{'west': -92.088777203125,
  'east': -83.936921734375,
  'south': 32.768894735707235,
  'north': 34.35501066067016},
 {'west': -92.088777203125,
  'east': -83.936921734375,
  'south': 34.35501066067016,
  'north': 35.941126585633086},
 {'west': -92.088777203125,
  'east': -83.936921734375,
  'south': 35.941126585633086,
  'north': 37.52724251059601},
 {'west': -83.936921734375,
  'east': -75.785066265625,
  'south': 32.768894735707235,
  'north': 34.35501066067016},
 {'west': -83.936921734375,
  'east': -75.785066265625,
  'south': 34.35501066067016,
  'north': 35.941126585633086},
 {'west': -83.936921734375,
  'east': -75.785066265625,
  'south': 35.941126585633086,
  'north': 37.52724251059601},
 {'west': -75.785066265625,
  'east': -67.633210796875,
  'south': 32.768894735707235,
  'north': 34.35501066067016},
 {'west': -75.785066265625,
  'east': -67.633210796875,
  'south': 34.35501066067016,
  'north': 35.941126585633086},
 {'west': -75.785066265625,
  'east': -67.633210796875,


In [98]:
responses = []
json_payloads = []
for i in splitted_coords:
    j_data = {
        'searchQueryState': {
            'pagination': {
                'currentPage': 21,
            },
            'isMapVisible': True,
            'mapBounds':i,
            'regionSelection': [
                {
                    'regionId': 36,
                    'regionType': 2,
                },
            ],
            'filterState': {
                'sortSelection': {
                    'value': 'globalrelevanceex',
                },
                'isForSaleByAgent': {
                    'value': False,
                },
                'isForSaleByOwner': {
                    'value': False,
                },
                'isNewConstruction': {
                    'value': False,
                },
                'isComingSoon': {
                    'value': False,
                },
                'isAuction': {
                    'value': False,
                },
                'isForSaleForeclosure': {
                    'value': False,
                },
                'isRecentlySold': {
                    'value': True,
                },
            },
            'isListVisible': True,
            'mapZoom': 6,
        },
        'wants': {
            'cat1': [
                'listResults',
                'mapResults',
            ],
        },
        'requestId': 11,
        'isDebugRequest': False,
    }
    json_payloads.append(j_data)
len(json_payloads)

9

In [100]:
import re
import requests
from curl_cffi import requests as curl_r
import json
import pandas as pd
import time, random

cookies = {
    '_hp2_id.1215457233': '%7B%22userId%22%3A%222230556399004484%22%2C%22pageviewId%22%3A%227996225023036108%22%2C%22sessionId%22%3A%222432367684716351%22%2C%22identity%22%3Anull%2C%22trackerVersion%22%3A%224.0%22%7D',
    'zguid': '24|%24bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0',
    'zjs_anonymous_id': '%22bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0%22',
    'zjs_user_id': 'null',
    'zg_anonymous_id': '%224e64cb82-f529-43d3-b1d0-bc0fbc6e91f9%22',
    '_pxvid': '37af2b21-cf4e-11ef-8b7c-f8ee72dc674e',
    'zgsession': '1|c6136cdb-2a3d-43c4-a13e-30f674889b3f',
    'pxcts': '7ebf61ea-cf7d-11ef-bd49-814e23b88f59',
    'JSESSIONID': '6701FC2E2A7103B157F9A77E282C8713',
    'AWSALB': '6mRIvoZkCIKT6Z7SBTr64wDVFtIpevFu+h7zIUGtuGS+OzK6aWYHmlP+o1dMAicFA/CkHxanHT1Z5LZfJ2dUyaTbaKL52rAij2CnxmoBAlW8HwjwQvvp60SxIDcb',
    'AWSALBCORS': '6mRIvoZkCIKT6Z7SBTr64wDVFtIpevFu+h7zIUGtuGS+OzK6aWYHmlP+o1dMAicFA/CkHxanHT1Z5LZfJ2dUyaTbaKL52rAij2CnxmoBAlW8HwjwQvvp60SxIDcb',
    '_px3': 'cac93eabba8935c2ec8af0a9a2a09406e6106b40c1683a92437ce87cabe5b5ef:UpNf/MXfpWi4WU8GIezSkqrZywDgcaVvTHcElEMsjWeZ/3npZvR8GEtC7V9g21+9MpziFD6VvxW+qlwCXQbvUg==:1000:3iWkVXZfmCXyphRkM9dPBriCPx4t3lk5PyDID6T4n+PhhQb2WnKlZ8XSsXtyRNdnPmXQwkKZlevJJDWbOSbrdoH3JNuWyqoAbTrOqt/7W0sjOQNSyfd6eEWfFNbJOYB1XHI4F4rH4oftUB7jZd+V+3gF9cY39Ym0EXICoGVvxrobx4KXmIiQeRm6n0C7TchP+OcHQfKXl07z22Om7PIFWhaCoRVo5grazf4EmwCPp2c=',
    'search': '6|1739125908779%7Crect%3D38.71989219561046%2C-67.633210796875%2C31.48498897341223%2C-92.088777203125%26rid%3D36%26disp%3Dmap%26mdm%3Dauto%26p%3D7%26listPriceActive%3D1%26fs%3D0%26fr%3D0%26mmm%3D0%26rs%3D1%26singlestory%3D0%26housing-connector%3D0%26parking-spots%3Dnull-%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26showcase%3D0%26featuredMultiFamilyBuilding%3D0%26onlyRentalStudentHousingType%3D0%26onlyRentalIncomeRestrictedHousingType%3D0%26onlyRentalMilitaryHousingType%3D0%26onlyRentalDisabledHousingType%3D0%26onlyRentalSeniorHousingType%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0936%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09',
}
headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'origin': 'https://www.zillow.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://www.zillow.com/nc/sold/7_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A7%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-92.088777203125%2C%22east%22%3A-67.633210796875%2C%22south%22%3A32.768894735707235%2C%22north%22%3A37.52724251059601%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A36%2C%22regionType%22%3A2%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%7D',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}
json_data = {
    'searchQueryState': {
        'pagination': {
            'currentPage': 21,
        },
        'isMapVisible': True,
        'mapBounds': {
            'west': -92.088777203125,
            'east': -79.860994,
            'south': 32.768894735707235,
            'north': 35.14806862315162
        },
        'regionSelection': [
            {
                'regionId': 36,
                'regionType': 2,
            },
        ],
        'filterState': {
            'sortSelection': {
                'value': 'globalrelevanceex',
            },
            'isForSaleByAgent': {
                'value': False,
            },
            'isForSaleByOwner': {
                'value': False,
            },
            'isNewConstruction': {
                'value': False,
            },
            'isComingSoon': {
                'value': False,
            },
            'isAuction': {
                'value': False,
            },
            'isForSaleForeclosure': {
                'value': False,
            },
            'isRecentlySold': {
                'value': True,
            },
        },
        'isListVisible': True,
        'mapZoom': 6,
    },
    'wants': {
        'cat1': [
            'listResults',
            'mapResults',
        ],
    },
    'requestId': 11,
    'isDebugRequest': False,
}


In [101]:
for j in json_payloads:
    response = requests.put('https://www.zillow.com/async-create-search-page-state', cookies=cookies, headers=headers, json=j)
    try: 
        print("Found items:", len(response.json()['cat1']['searchResults']['mapResults']))
        responses.extend(response.json()['cat1']['searchResults']['mapResults'])
        time.sleep(random.randint(2, 6))
    except KeyError:
        print("Non-existing key. Available keys are:\n", response.json().keys())    
print(len(responses))

Found items: 0
Found items: 500
Found items: 0
Found items: 500
Found items: 500
Found items: 500
Found items: 0
Found items: 500
Found items: 500
3000


In [115]:
df = pd.DataFrame(responses)
df

,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,...,marketingTreatments,info2String,info6String,plid,communityName,builderName,isCdpResult,isComingSoonCommunity,style,buildingName
0,123771975,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/5aa720668de...,True,/homedetails/14-Wells-Rd-Brasstown-NC-28902/12...,SOLD,Sold,"$450,000",$450K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,230500113,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/11e1b47cc95...,True,/homedetails/544-Cobb-Cir-Murphy-NC-28906/2305...,SOLD,Sold,"$310,000",$310K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,230509452,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/67d1e449729...,True,/homedetails/130-Springdale-St-Murphy-NC-28906...,SOLD,Sold,"$175,000",$175K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,241900910,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/4420493cd85...,True,/homedetails/1095-Lonesome-Pine-Rd-Murphy-NC-2...,SOLD,Sold,"$305,000",$305K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,247285902,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/7f4a5656837...,True,/homedetails/365-Virginia-Ln-Murphy-NC-28906/2...,SOLD,Sold,"$335,000",$335K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,228632585,RecentlySold,Non Owner Occupied,https://maps.googleapis.com/maps/api/staticmap...,NaN,/homedetails/2421-N-Virginia-Dare-Trl-Kill-Dev...,SOLD,Sold,$1.30M,$1.30M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2996,348544520,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/9299adfa842...,True,/homedetails/901-W-4th-St-LOT-29-Kill-Devil-Hi...,SOLD,Sold,"$399,900",$400K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2997,349431010,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/7a47fb1f02b...,True,/homedetails/4602-S-Blue-Marlin-Way-78-Nags-He...,SOLD,Sold,"$821,000",$821K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,402657548,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/7ef7e881788...,True,/homedetails/114-Outrigger-Dr-LOT-21-Kill-Devi...,SOLD,Sold,"$399,900",$400K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
to_keep = ['address', 'price', 'beds', 'baths', 'area', 'detailUrl']
df = df[to_keep]
df

,address,price,beds,baths,area,detailUrl
0,"14 Wells Rd, Brasstown, NC 28902","$450,000",3.0,4.0,NaN,/homedetails/14-Wells-Rd-Brasstown-NC-28902/12...
1,"544 Cobb Cir, Murphy, NC 28906","$310,000",3.0,2.0,NaN,/homedetails/544-Cobb-Cir-Murphy-NC-28906/2305...
2,"130 Springdale St, Murphy, NC 28906","$175,000",4.0,3.0,2514.0,/homedetails/130-Springdale-St-Murphy-NC-28906...
3,"1095 Lonesome Pine Rd, Murphy, NC 28906","$305,000",2.0,2.0,1008.0,/homedetails/1095-Lonesome-Pine-Rd-Murphy-NC-2...
4,"365 Virginia Ln, Murphy, NC 28906","$335,000",2.0,2.0,NaN,/homedetails/365-Virginia-Ln-Murphy-NC-28906/2...
...,...,...,...,...,...,...
2995,"2421 N Virginia Dare Trl, Kill Devil Hills, NC...",$1.30M,4.0,2.5,1976.0,/homedetails/2421-N-Virginia-Dare-Trl-Kill-Dev...
2996,"901 W 4th St LOT 29, Kill Devil Hills, NC 27948","$399,900",3.0,2.0,1435.0,/homedetails/901-W-4th-St-LOT-29-Kill-Devil-Hi...
2997,"4602 S Blue Marlin Way #78, Nags Head, NC 27959","$821,000",3.0,2.0,1807.0,/homedetails/4602-S-Blue-Marlin-Way-78-Nags-He...
2998,"114 Outrigger Dr LOT 21, Kill Devil Hills, NC ...","$399,900",3.0,3.0,1376.0,/homedetails/114-Outrigger-Dr-LOT-21-Kill-Devi...


In [117]:
df = df.drop_duplicates()
df.shape

(2999, 6)

In [118]:
df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))
df

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_5456\3280207385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))


,address,price,beds,baths,area,detailUrl
0,"14 Wells Rd, Brasstown, NC 28902",450000,3.0,4.0,NaN,/homedetails/14-Wells-Rd-Brasstown-NC-28902/12...
1,"544 Cobb Cir, Murphy, NC 28906",310000,3.0,2.0,NaN,/homedetails/544-Cobb-Cir-Murphy-NC-28906/2305...
2,"130 Springdale St, Murphy, NC 28906",175000,4.0,3.0,2514.0,/homedetails/130-Springdale-St-Murphy-NC-28906...
3,"1095 Lonesome Pine Rd, Murphy, NC 28906",305000,2.0,2.0,1008.0,/homedetails/1095-Lonesome-Pine-Rd-Murphy-NC-2...
4,"365 Virginia Ln, Murphy, NC 28906",335000,2.0,2.0,NaN,/homedetails/365-Virginia-Ln-Murphy-NC-28906/2...
...,...,...,...,...,...,...
2995,"2421 N Virginia Dare Trl, Kill Devil Hills, NC...",130,4.0,2.5,1976.0,/homedetails/2421-N-Virginia-Dare-Trl-Kill-Dev...
2996,"901 W 4th St LOT 29, Kill Devil Hills, NC 27948",399900,3.0,2.0,1435.0,/homedetails/901-W-4th-St-LOT-29-Kill-Devil-Hi...
2997,"4602 S Blue Marlin Way #78, Nags Head, NC 27959",821000,3.0,2.0,1807.0,/homedetails/4602-S-Blue-Marlin-Way-78-Nags-He...
2998,"114 Outrigger Dr LOT 21, Kill Devil Hills, NC ...",399900,3.0,3.0,1376.0,/homedetails/114-Outrigger-Dr-LOT-21-Kill-Devi...


In [119]:
df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")
df.loc[100, :]

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_5456\728619470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")


address                 161 Smokey Hollow Dr, Murphy, NC 28906
price                                                   414000
beds                                                       2.0
baths                                                      2.0
area                                                    1450.0
detailUrl    /homedetails/161-Smokey-Hollow-Dr-Murphy-NC-28...
link         https://www.zillow.com//homedetails/161-Smokey...
Name: 100, dtype: object

In [120]:
df = df.drop(columns='detailUrl')
df

,address,price,beds,baths,area,link
0,"14 Wells Rd, Brasstown, NC 28902",450000,3.0,4.0,NaN,https://www.zillow.com//homedetails/14-Wells-R...
1,"544 Cobb Cir, Murphy, NC 28906",310000,3.0,2.0,NaN,https://www.zillow.com//homedetails/544-Cobb-C...
2,"130 Springdale St, Murphy, NC 28906",175000,4.0,3.0,2514.0,https://www.zillow.com//homedetails/130-Spring...
3,"1095 Lonesome Pine Rd, Murphy, NC 28906",305000,2.0,2.0,1008.0,https://www.zillow.com//homedetails/1095-Lones...
4,"365 Virginia Ln, Murphy, NC 28906",335000,2.0,2.0,NaN,https://www.zillow.com//homedetails/365-Virgin...
...,...,...,...,...,...,...
2995,"2421 N Virginia Dare Trl, Kill Devil Hills, NC...",130,4.0,2.5,1976.0,https://www.zillow.com//homedetails/2421-N-Vir...
2996,"901 W 4th St LOT 29, Kill Devil Hills, NC 27948",399900,3.0,2.0,1435.0,https://www.zillow.com//homedetails/901-W-4th-...
2997,"4602 S Blue Marlin Way #78, Nags Head, NC 27959",821000,3.0,2.0,1807.0,https://www.zillow.com//homedetails/4602-S-Blu...
2998,"114 Outrigger Dr LOT 21, Kill Devil Hills, NC ...",399900,3.0,3.0,1376.0,https://www.zillow.com//homedetails/114-Outrig...


In [121]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')

df.dtypes

address     object
price      float64
beds       float64
baths      float64
area       float64
link        object
dtype: object

In [122]:
df = df.fillna('-')

In [123]:
df

,address,price,beds,baths,area,link
0,"14 Wells Rd, Brasstown, NC 28902",450000.0,3.0,4.0,-,https://www.zillow.com//homedetails/14-Wells-R...
1,"544 Cobb Cir, Murphy, NC 28906",310000.0,3.0,2.0,-,https://www.zillow.com//homedetails/544-Cobb-C...
2,"130 Springdale St, Murphy, NC 28906",175000.0,4.0,3.0,2514.0,https://www.zillow.com//homedetails/130-Spring...
3,"1095 Lonesome Pine Rd, Murphy, NC 28906",305000.0,2.0,2.0,1008.0,https://www.zillow.com//homedetails/1095-Lones...
4,"365 Virginia Ln, Murphy, NC 28906",335000.0,2.0,2.0,-,https://www.zillow.com//homedetails/365-Virgin...
...,...,...,...,...,...,...
2995,"2421 N Virginia Dare Trl, Kill Devil Hills, NC...",130.0,4.0,2.5,1976.0,https://www.zillow.com//homedetails/2421-N-Vir...
2996,"901 W 4th St LOT 29, Kill Devil Hills, NC 27948",399900.0,3.0,2.0,1435.0,https://www.zillow.com//homedetails/901-W-4th-...
2997,"4602 S Blue Marlin Way #78, Nags Head, NC 27959",821000.0,3.0,2.0,1807.0,https://www.zillow.com//homedetails/4602-S-Blu...
2998,"114 Outrigger Dr LOT 21, Kill Devil Hills, NC ...",399900.0,3.0,3.0,1376.0,https://www.zillow.com//homedetails/114-Outrig...


In [124]:
df.to_csv('final_homes.csv', index=False)